In [1]:
import pandas as pd
import numpy as np
import featuretools as ft
%load_ext autoreload
%autoreload 2

num_rows = 10000
int_cols = 10
float_cols = 10
id_columns = 4
num_groups = 1000

In [2]:
try:
    es = ft.read_entityset("groupby_benchmark/")
except:
    print("creating entityset")
    int_df = pd.DataFrame(np.random.randint(0,100,size=(num_rows, int_cols))).add_suffix("_int")
    float_df = pd.DataFrame(np.random.randint(0,100000,size=(num_rows, float_cols)) / 100).add_suffix("_float")
    id_df = pd.DataFrame(np.random.randint(0, num_groups,size=(num_rows, id_columns)), dtype=str).add_suffix("_id")
    df = pd.concat([int_df, float_df, id_df], axis=1)


    variable_types = {}

    for col in id_df.columns:
        variable_types[col] = ft.variable_types.Id

    es = ft.EntitySet()
    es.entity_from_dataframe(entity_id="entity",
                             dataframe=df,
                             index="index",
                             variable_types=variable_types,
                             make_index=True)
    es.to_csv("groupby_benchmark", compression="gzip")
    
es

Entityset: None
  Entities:
    entity [Rows: 10000, Columns: 25]
  Relationships:
    No relationships

In [3]:
fl = ft.dfs(target_entity="entity",
       entityset=es,
       groupby_trans_primitives=["cum_sum", "cum_max", "cum_min"],
       max_depth=1,
       features_only=True,
       verbose=True)

len(fl)

Built 264 features


264

In [8]:
fm = ft.calculate_feature_matrix(entityset=es,
                                 features=fl,
                                 chunk_size=num_rows,
                                 verbose=True)
fm.head()

Elapsed: 01:35 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks


,0_int,1_int,2_int,3_int,4_int,5_int,6_int,7_int,8_int,9_int,...,CUM_MIN(2_int) by 0_id,CUM_MIN(1_float) by 0_id,CUM_MIN(2_int) by 2_id,CUM_MIN(1_int) by 3_id,CUM_MIN(3_int) by 3_id,CUM_MIN(6_float) by 3_id,CUM_MIN(3_float) by 1_id,CUM_MIN(1_float) by 2_id,CUM_MIN(5_int) by 2_id,CUM_MIN(6_int) by 3_id
index,,,,,,,,,,,,,,,,,,,,,
0,63,28,94,16,24,89,42,7,56,76,...,94.0,266.50,94.0,28.0,16.0,967.80,327.29,266.50,89.0,42.0
1,7,45,85,73,85,59,8,10,28,96,...,85.0,637.22,85.0,45.0,73.0,822.66,398.90,637.22,59.0,8.0
2,6,34,54,18,54,6,27,55,15,94,...,54.0,417.94,54.0,34.0,18.0,639.18,172.39,417.94,6.0,27.0
3,33,76,46,70,71,8,23,69,74,9,...,46.0,428.99,46.0,76.0,70.0,151.18,150.77,428.99,8.0,23.0
4,19,27,45,16,97,0,32,84,51,37,...,45.0,806.16,45.0,27.0,16.0,27.21,529.70,806.16,0.0,32.0


In [ ]:
TEST_NAME = "by_features_2.csv"
fm.to_csv(TEST_NAME)

In [5]:
# check that it gives same results
truth = pd.read_csv("master.csv")
test = pd.read_csv(TEST_NAME)[truth.columns]

truth.equals(test)

True